In [1]:
#pip install tensorflow

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)


In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 3:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

3900


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## RANDOM FOREST MODEL

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [18]:
param_grid = {
    'n_estimators': [80, 100, 120],
    'max_depth': [None, 15, 20, 25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [ 'sqrt', 'log2'],
    'bootstrap': [False]
}


In [19]:
# Create the Random Forest model
rf = RandomForestClassifier()

In [20]:
# Create the GridSearchCV object
grid_search = GridSearchCV(rf, param_grid, cv=5, verbose=2, error_score='raise')

In [ ]:
# Fit the GridSearchCV object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=80; total time= 5.2min
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=80; total time= 5.8min


In [ ]:
# Get the best estimator and evaluate it
best_rf = grid_search.best_estimator_
y_test_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Random Forest Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
df = pd.DataFrame(grid_search.cv_results_)
df

In [ ]:
df[['param_n_estimators', 'params', 'mean_test_score']]

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [ ]:
# import matplotlib.pyplot as plt

# k_values = [ 3, 5, 7, 9, 11]  # List of K values
# accuracies = [ 0.943803, 0.942521, 0.941880, 0.941453, 0.939530]  # Corresponding accuracies

# plt.figure(figsize=(8, 6))
# plt.plot(k_values, accuracies, marker='o')
# plt.xlabel('K (Number of Neighbors)')
# plt.ylabel('Accuracy')
# plt.title('Accuracy vs. K')
# plt.grid(True)
# plt.show()

In [ ]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# y_test_pred = grid_search.predict(X_test)
# # Calculate the confusion matrix
# cm = confusion_matrix(y_test, y_test_pred)

# # Create a heatmap of the confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# plt.title("Confusion Matrix")
# plt.show()

In [ ]:
# from sklearn.model_selection import cross_val_score

# cv_scores = cross_val_score(grid_search, X, y, cv=5)  # Perform 5-fold cross-validation

# plt.figure(figsize=(8, 6))
# plt.bar(range(1, 6), cv_scores)
# plt.xlabel('Fold')
# plt.ylabel('Accuracy')
# plt.title('Cross-Validation Results')
# plt.grid(axis='y')
# plt.show()